In [26]:
import numpy as np
from sklearn.feature_extraction import FeatureHasher
import pandas as pd
import os

# Prepare Training data

In [2]:
# local_raw_data = '../data/ordered_ex1_train.csv'
local_raw_data = 'ordered_ex1_train.csv'

In [3]:
raw_data_df = pd.read_csv(local_raw_data)

In [4]:
# target (percentage of having adverse events) = number_of_sae_subjects / enrollment
raw_data_df['target'] = raw_data_df['number_of_sae_subjects'] / raw_data_df['enrollment']

In [5]:
# drop number_of_sae_subjects and enrollment
raw_data_df.drop(raw_data_df.columns[[7,8]], axis=1, inplace=True)

In [6]:
# convert dataframe to ndarray
raw_data = raw_data_df.as_matrix()

In [7]:
# map has_us_facilities value ('t','f') -> (1, 0)
raw_data[raw_data[:,4] == 't',4]= 1
raw_data[raw_data[:,4] == 'f',4]= 0

## Feature hashing

In [8]:
def merge_category(id_cat):
    output = []
    prev_id = None
    tmp = {}
    for nct_id, cat in id_cat:
        if prev_id is None or prev_id == nct_id:
            if prev_id is None:
                prev_id = nct_id
            tmp[cat] = 1
        else:
            output.append(tmp)
            # reset
            tmp = {}
            tmp[cat] = 1
            prev_id = nct_id
    output.append(tmp)
    print('output number of merged unique ids: {}'.format(len(output)))
    return output

### Convert conditions, interventions and countries to hash feature

In [9]:
unique_ids = np.unique(raw_data[:,0])
number_of_uniqueID = unique_ids.shape[0]
print('number of unique nct_ids: {}'.format(number_of_uniqueID))

number of unique nct_ids: 16677


In [10]:
conditions = raw_data[:,[0,1]]
interventions = raw_data[:,[0,2]]
countries = raw_data[:,[0,5]]

> TODO: draw frequency distribution of the three features

In [11]:
interventions[:,1]

array(['ganciclovir', 'ganciclovir', 'ganciclovir', ..., 'oxycodone',
       'fentanyl', 'adenosine'], dtype=object)

In [12]:
# preprocess the high dimentional features before feed into feature hasher
merged_conditions = merge_category(conditions)
merged_interventions = merge_category(interventions)
merged_countries = merge_category(countries)

output number of merged unique ids: 16677
output number of merged unique ids: 16677
output number of merged unique ids: 16677


In [13]:
number_of_conditions = np.unique(conditions[:,1]).shape[0]
print('number of unique conditions: {}'.format(number_of_conditions))

number_of_interventions = np.unique(interventions[:,1]).shape[0]
print('number of unique interventions: {}'.format(number_of_interventions))

number_of_countries = np.unique(countries[:,1]).shape[0]
print('number of unique countries: {}'.format(number_of_countries))

number of unique conditions: 1909
number of unique interventions: 1846
number of unique countries: 142


In [14]:
# feature hasher
conditions_hasher = FeatureHasher(n_features=int(number_of_conditions * 0.2),
                                                             non_negative=True,input_type='dict')
interventions_hasher = FeatureHasher(n_features=int(number_of_interventions * 0.2),
                                                             non_negative=True,input_type='dict')
countries_hasher = FeatureHasher(n_features=int(number_of_countries),
                                                             non_negative=True,input_type='dict')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [15]:
# apply feature hashing
conditions_feature = conditions_hasher.fit_transform(merged_conditions).toarray()
print('conditions_feature shape: {}'.format(conditions_feature.shape))

interventions_feature = interventions_hasher.fit_transform(merged_interventions).toarray()
print('interventions_feature shape: {}'.format(interventions_feature.shape))

countries_feature = countries_hasher.fit_transform(merged_countries).toarray()
print('countries_feature shape: {}'.format(countries_feature.shape))

conditions_feature shape: (16677, 381)
interventions_feature shape: (16677, 369)
countries_feature shape: (16677, 142)


### Appending the hashed feature to training data

In [17]:
raw_data[0]

array(['NCT00000143', 'hiv infections', 'ganciclovir', 19, 1,
       'United States', 1, 0.0], dtype=object)

In [18]:
# drop the old conditions, interventions and countries
prev_id = None
new_data = []
tmp = []
idx = 0
for data in raw_data:
    cur_id = data[0]
    if prev_id is None or cur_id != prev_id:
        tmp.append(cur_id)
        tmp.append(data[3])
        tmp.append(data[4])
        tmp.append(data[6])
        
        tmp += conditions_feature[idx].tolist()
        tmp += interventions_feature[idx].tolist()
        tmp += countries_feature[idx].tolist()
        
        tmp.append(data[7])
        
        new_data.append(tmp)
        
        # update
        prev_id = cur_id
        tmp = []
        idx += 1

# new data shape: nct_id, number_of_facilities, has_us_facility, 
# number_of_sponsors, conditions_features, interventions_features, contries_features, percentage_of_adverse_event (target)
new_data = np.array(new_data)
print('reconstructed training data shape: {}'.format(new_data.shape))

reconstructed training data shape: (16677, 897)


> note: remove the first nct_id column before send to model

In [19]:
# randomly shuffle the data before categorization
np.random.shuffle(new_data)

In [20]:
train_size = int(new_data.shape[0] * 0.7)
train_features  = new_data[:train_size, :-1]
train_features

array([['NCT01349959', '7', '1', ..., '0.0', '0.0', '0.0'],
       ['NCT00703937', '1', '1', ..., '0.0', '0.0', '0.0'],
       ['NCT01575899', '1', '0', ..., '0.0', '0.0', '0.0'],
       ...,
       ['NCT00805142', '22', '0', ..., '0.0', '0.0', '0.0'],
       ['NCT01733316', '13', '1', ..., '0.0', '1.0', '0.0'],
       ['NCT01027351', '1', '0', ..., '0.0', '0.0', '0.0']], dtype='<U22')

In [21]:
train_size = int(new_data.shape[0] * 0.7)
train_features  = new_data[:train_size, :-1]
train_labels = new_data[:train_size, -1]
print('train_features shape: {}'.format(train_features.shape))

validation_size = int(new_data.shape[0] * 0.2)
validation_features = new_data[train_size:train_size + validation_size, :-1]
validation_labels = new_data[train_size:train_size + validation_size, -1]
print('validation_features shape: {}'.format(validation_features.shape))

test_features = new_data[train_size + validation_size:, :-1]
test_labels = new_data[train_size + validation_size:, -1]
print('test_features shape: {}'.format(test_features.shape))

train_features shape: (11673, 896)
validation_features shape: (3335, 896)
test_features shape: (1669, 896)


In [33]:
feature_dim = train_features[:,1:].shape[1]
print('feature dimention: {}'.format(feature_dim))

feature dimention: 895


### Data conversion

convert the numpy array into recordIO-protobuf or CSV format that can be used by sagemaker linear_learner
model specs: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

Since algorithms have particular input and output requirements, converting the dataset is also part of the process that a data scientist goes through prior to initiating training. In this particular case, the Amazon SageMaker implementation of Linear Learner takes recordIO-wrapped protobuf, where the data we have today is a pickle-ized numpy array on disk.

Most of the conversion effort is handled by the Amazon SageMaker Python SDK, imported as `sagemaker` below.

In [35]:
import boto3
import re
import sagemaker.amazon.common as smac
import io
from sagemaker import get_execution_role
import sagemaker

bucket = 'sagemaker-tiber-solution'
prefix = 'logistic-regression/yangz5'
s3_train_key = '{}/train/recordio-pb-data'.format(prefix)
s3_train_path = os.path.join('s3://', bucket, s3_train_key)

# Define IAM role
role = get_execution_role()

In [30]:
# traing data should be excluding the first nct_id column 
vectors = np.array([t.tolist() for t in train_features[:,1:]]).astype('float32')
labels = np.array([t.tolist() for t in train_labels]).astype('float32')
buf = io.BytesIO()
smac.write_numpy_to_dense_tensor(buf, vectors, labels)
buf.seek(0)
boto3.resource('s3').Bucket(bucket).Object(s3_train_key).upload_fileobj(buf)

Wrapping the model training setup in a convenience function that takes in the S3 location of the training data, the model hyperparameters that define our training job, and the S3 output path for model artifacts.  Inside the function, we'll hardcode the algorithm container, the number and type of EC2 instances to train on, and the input and output data formats.

In [32]:
def predictor_from_hyperparams(s3_train_data, hyperparams, output_path):
    """
    Create an Estimator from the given hyperparams, fit to training data, and return a deployed predictor
    """
    # specify algorithm containers and instantiate an Estimator with given hyperparams
    containers = {
        'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest',
        'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/linear-learner:latest',
        'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/linear-learner:latest',
        'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/linear-learner:latest'}
    linear = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
        role,
        train_instance_count=1,
        train_instance_type='ml.m4.xlarge',
        output_path=output_path,
        sagemaker_session=sagemaker.Session())
    linear.set_hyperparameters(**hyperparams)
    # train model
    linear.fit({'train': s3_train_data})
#     # deploy a predictor
#     linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
#     linear_predictor.content_type = 'text/csv'
#     linear_predictor.serializer = csv_serializer
#     linear_predictor.deserializer = json_deserializer
#     return linear_predictor

    # return the trained model, for now
    return linear

In [37]:
# Training a binary classifier with default settings: logistic regression
defaults_hyperparams = {
    'feature_dim': feature_dim,
    'predictor_type': 'regressor',
    'epochs': 10
}
defaults_output_path = 's3://{}/{}/defaults/output'.format(bucket, prefix)
defaults_predictor = predictor_from_hyperparams(s3_train_path, defaults_hyperparams, defaults_output_path)

INFO:sagemaker:Creating training-job with name: linear-learner-2018-04-11-16-39-20-286


..................................................................
Docker entrypoint called with argument(s): train
[04/11/2018 16:44:44 INFO 139860162471744] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'10', u'init_bias': u'0.0', u'lr_scheduler_factor': u'0.99', u'num_calibration_samples': u'10000000', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0.5', u'bias_lr_mult': u'10', u'lr_scheduler_step': u'100', u'init_method': u'uniform', u'init_sigma': u'0.01', u'lr_scheduler_minimum_lr': u'0.00001', u'target_recall': u'0.8', u'num_models': u'32', u'early_stopping_patience': u'3', u'momentum': u'0.0', u'unbias_label': u'auto', u'wd': u'0.0', u'optimizer': u'adam', u'early_stopping_tolerance': u'0.001', u'learning_rate': u'auto', u'_kvstore': u'auto', u'normalize_data': u'true', u'binary_classifier_model_sel

#metrics {"Metrics": {"training_mse": {"count": 1, "max": 0.6891122963645242, "sum": 0.6891122963645242, "min": 0.6891122963645242}}, "EndTime": 1523465087.327766, "Dimensions": {"model": 0, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1523465087.327691}

#metrics {"Metrics": {"training_mse": {"count": 1, "max": 0.6891054271351207, "sum": 0.6891054271351207, "min": 0.6891054271351207}}, "EndTime": 1523465087.327848, "Dimensions": {"model": 1, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1523465087.327834}

#metrics {"Metrics": {"training_mse": {"count": 1, "max": 0.6906183471679688, "sum": 0.6906183471679688, "min": 0.6906183471679688}}, "EndTime": 1523465087.327899, "Dimensions": {"model": 2, "Host": "algo-1", "Operation": "training", "Algorithm": "Linear Learner", "epoch": 6}, "StartTime": 1523465087.327886}

#metrics {"Metrics": {"training_mse": {"count": 1, "max": 0.6905177

===== Job Complete =====
Billable seconds: 172


In [ ]:
logistic_predictor = defaults_predictor.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [40]:
from sagemaker.predictor import csv_serializer, json_deserializer
logistic_predictor.content_type = 'text/csv'
logistic_predictor.serializer = csv_serializer
logistic_predictor.deserializer = json_deserializer

In [90]:
result = []

for idx, train_line in enumerate(test_features):
    prediction = logistic_predictor.predict(train_line[1:].reshape(1, -1))
    result.append((train_line[0], prediction['predictions'][0]['score'], float(test_labels[idx])))
#     print('idx: {}, nct_id: {}, prediction: {}, label: {}'.format(idx, train_line[0], prediction['predictions'][0]['score'], train_labels[idx]))

{'predictions': [{'score': 1.4546172618865967}]}
idx: 0, nct_id: NCT01349959, prediction: 1.4546172618865967, label: 0.425
{'predictions': [{'score': 0.5182403326034546}]}
idx: 1, nct_id: NCT00703937, prediction: 0.5182403326034546, label: 0.08192090395480225
{'predictions': [{'score': 0.7158364057540894}]}
idx: 2, nct_id: NCT01575899, prediction: 0.7158364057540894, label: 0.009615384615384616
{'predictions': [{'score': 0.03394302725791931}]}
idx: 3, nct_id: NCT00621517, prediction: 0.03394302725791931, label: 0.03333333333333333
{'predictions': [{'score': 0.6007420420646667}]}
idx: 4, nct_id: NCT01100853, prediction: 0.6007420420646667, label: 0.32
{'predictions': [{'score': 0.29297369718551636}]}
idx: 5, nct_id: NCT01444651, prediction: 0.29297369718551636, label: 0.0
{'predictions': [{'score': 0.23890921473503113}]}
idx: 6, nct_id: NCT02114931, prediction: 0.23890921473503113, label: 0.22269807280513917
{'predictions': [{'score': -0.05315151810646057}]}
idx: 7, nct_id: NCT00885079,

{'predictions': [{'score': 0.03668370842933655}]}
idx: 66, nct_id: NCT00985010, prediction: 0.03668370842933655, label: 0.0
{'predictions': [{'score': 1.4654052257537842}]}
idx: 67, nct_id: NCT01317615, prediction: 1.4654052257537842, label: 1.9387755102040816
{'predictions': [{'score': -0.047054439783096313}]}
idx: 68, nct_id: NCT00427700, prediction: -0.047054439783096313, label: 0.0
{'predictions': [{'score': -0.0583270788192749}]}
idx: 69, nct_id: NCT01017263, prediction: -0.0583270788192749, label: 0.0
{'predictions': [{'score': -0.10395672917366028}]}
idx: 70, nct_id: NCT00468481, prediction: -0.10395672917366028, label: 0.01818181818181818
{'predictions': [{'score': 0.47391563653945923}]}
idx: 71, nct_id: NCT00986154, prediction: 0.47391563653945923, label: 0.5062711046792089
{'predictions': [{'score': 0.2617266774177551}]}
idx: 72, nct_id: NCT00632970, prediction: 0.2617266774177551, label: 0.0
{'predictions': [{'score': 0.3707788586616516}]}
idx: 73, nct_id: NCT00141726, predi

{'predictions': [{'score': 0.5711166858673096}]}
idx: 128, nct_id: NCT01071070, prediction: 0.5711166858673096, label: 0.027777777777777776
{'predictions': [{'score': 0.6696575880050659}]}
idx: 129, nct_id: NCT01738646, prediction: 0.6696575880050659, label: 0.8541666666666666
{'predictions': [{'score': 1.191772699356079}]}
idx: 130, nct_id: NCT00581919, prediction: 1.191772699356079, label: 0.84375
{'predictions': [{'score': 0.879747748374939}]}
idx: 131, nct_id: NCT01042509, prediction: 0.879747748374939, label: 1.3333333333333333
{'predictions': [{'score': 0.8589233756065369}]}
idx: 132, nct_id: NCT00672243, prediction: 0.8589233756065369, label: 0.5
{'predictions': [{'score': -0.6865968108177185}]}
idx: 133, nct_id: NCT00676130, prediction: -0.6865968108177185, label: 0.013071895424836602
{'predictions': [{'score': 0.9139930605888367}]}
idx: 134, nct_id: NCT00108069, prediction: 0.9139930605888367, label: 0.9302325581395349
{'predictions': [{'score': 0.21574559807777405}]}
idx: 135

{'predictions': [{'score': 0.056925803422927856}]}
idx: 208, nct_id: NCT00106106, prediction: 0.056925803422927856, label: 0.03571428571428571
{'predictions': [{'score': 0.238401859998703}]}
idx: 209, nct_id: NCT01188564, prediction: 0.238401859998703, label: 0.02666666666666667
{'predictions': [{'score': 0.050325408577919006}]}
idx: 210, nct_id: NCT01411774, prediction: 0.050325408577919006, label: 0.0
{'predictions': [{'score': 0.522557258605957}]}
idx: 211, nct_id: NCT00788593, prediction: 0.522557258605957, label: 0.18292682926829268
{'predictions': [{'score': 0.30909085273742676}]}
idx: 212, nct_id: NCT00535782, prediction: 0.30909085273742676, label: 0.6439393939393939
{'predictions': [{'score': 0.06693802773952484}]}
idx: 213, nct_id: NCT00343915, prediction: 0.06693802773952484, label: 0.03745318352059925
{'predictions': [{'score': 0.37124693393707275}]}
idx: 214, nct_id: NCT00844714, prediction: 0.37124693393707275, label: 0.0
{'predictions': [{'score': 0.16901680827140808}]}


{'predictions': [{'score': 0.044431865215301514}]}
idx: 269, nct_id: NCT01337297, prediction: 0.044431865215301514, label: 0.0
{'predictions': [{'score': 0.7967870831489563}]}
idx: 270, nct_id: NCT01085331, prediction: 0.7967870831489563, label: 1.5625
{'predictions': [{'score': 0.10628631711006165}]}
idx: 271, nct_id: NCT00826943, prediction: 0.10628631711006165, label: 0.0
{'predictions': [{'score': 0.47470948100090027}]}
idx: 272, nct_id: NCT00661427, prediction: 0.47470948100090027, label: 1.540983606557377
{'predictions': [{'score': 0.1277557909488678}]}
idx: 273, nct_id: NCT00959192, prediction: 0.1277557909488678, label: 0.0
{'predictions': [{'score': 0.027279138565063477}]}
idx: 274, nct_id: NCT01155999, prediction: 0.027279138565063477, label: 0.0
{'predictions': [{'score': 0.5507966876029968}]}
idx: 275, nct_id: NCT01280981, prediction: 0.5507966876029968, label: 0.041666666666666664
{'predictions': [{'score': 0.6560589671134949}]}
idx: 276, nct_id: NCT01459653, prediction: 0

...

{'predictions': [{'score': -0.23792213201522827}]}
idx: 3434, nct_id: NCT00304265, prediction: -0.23792213201522827, label: 0.0
{'predictions': [{'score': 0.3708491921424866}]}
idx: 3435, nct_id: NCT01652573, prediction: 0.3708491921424866, label: 0.0
{'predictions': [{'score': 0.5599980354309082}]}
idx: 3436, nct_id: NCT00772668, prediction: 0.5599980354309082, label: 0.6666666666666666
{'predictions': [{'score': 0.14026106894016266}]}
idx: 3437, nct_id: NCT01290679, prediction: 0.14026106894016266, label: 0.15776081424936386
{'predictions': [{'score': 0.17615842819213867}]}
idx: 3438, nct_id: NCT01166958, prediction: 0.17615842819213867, label: 0.0
{'predictions': [{'score': 0.061321407556533813}]}
idx: 3439, nct_id: NCT00303823, prediction: 0.061321407556533813, label: 0.02040816326530612
{'predictions': [{'score': 0.007159769535064697}]}
idx: 3440, nct_id: NCT00422162, prediction: 0.007159769535064697, label: 0.0855457227138643
{'predictions': [{'score': 0.19456058740615845}]}
idx:

{'predictions': [{'score': 1.4166903495788574}]}
idx: 3512, nct_id: NCT01984892, prediction: 1.4166903495788574, label: 0.0
{'predictions': [{'score': -0.014292150735855103}]}
idx: 3513, nct_id: NCT00940108, prediction: -0.014292150735855103, label: 0.03783783783783784
{'predictions': [{'score': 0.05092443525791168}]}
idx: 3514, nct_id: NCT00402194, prediction: 0.05092443525791168, label: 0.0
{'predictions': [{'score': 0.6343109607696533}]}
idx: 3515, nct_id: NCT00506415, prediction: 0.6343109607696533, label: 0.6502525252525253
{'predictions': [{'score': 0.06546521186828613}]}
idx: 3516, nct_id: NCT02258334, prediction: 0.06546521186828613, label: 0.009615384615384616
{'predictions': [{'score': -0.05659067630767822}]}
idx: 3517, nct_id: NCT02128490, prediction: -0.05659067630767822, label: 0.07407407407407407
{'predictions': [{'score': 0.46146154403686523}]}
idx: 3518, nct_id: NCT02604173, prediction: 0.46146154403686523, label: 0.0
{'predictions': [{'score': 0.118227019906044}]}
idx:

{'predictions': [{'score': 0.39075836539268494}]}
idx: 3590, nct_id: NCT00195702, prediction: 0.39075836539268494, label: 1.789983844911147
{'predictions': [{'score': 0.14279088377952576}]}
idx: 3591, nct_id: NCT01596231, prediction: 0.14279088377952576, label: 0.0
{'predictions': [{'score': -0.12152588367462158}]}
idx: 3592, nct_id: NCT00609674, prediction: -0.12152588367462158, label: 0.0
{'predictions': [{'score': 0.029803171753883362}]}
idx: 3593, nct_id: NCT00831428, prediction: 0.029803171753883362, label: 0.0
{'predictions': [{'score': 0.10019263625144958}]}
idx: 3594, nct_id: NCT01573767, prediction: 0.10019263625144958, label: 0.004319654427645789
{'predictions': [{'score': 0.480633944272995}]}
idx: 3595, nct_id: NCT00318474, prediction: 0.480633944272995, label: 0.021739130434782608
{'predictions': [{'score': 0.2062617540359497}]}
idx: 3596, nct_id: NCT00273910, prediction: 0.2062617540359497, label: 0.0
{'predictions': [{'score': 0.3223632872104645}]}
idx: 3597, nct_id: NCT0

{'predictions': [{'score': 0.6827464699745178}]}
idx: 3665, nct_id: NCT01036802, prediction: 0.6827464699745178, label: 0.6666666666666666
{'predictions': [{'score': 0.42850157618522644}]}
idx: 3666, nct_id: NCT02004990, prediction: 0.42850157618522644, label: 0.0
{'predictions': [{'score': 0.8334275484085083}]}
idx: 3667, nct_id: NCT00319735, prediction: 0.8334275484085083, label: 1.1219512195121952
{'predictions': [{'score': 0.5125675201416016}]}
idx: 3668, nct_id: NCT01125813, prediction: 0.5125675201416016, label: 0.125
{'predictions': [{'score': 0.11801734566688538}]}
idx: 3669, nct_id: NCT00785577, prediction: 0.11801734566688538, label: 0.03663003663003663
{'predictions': [{'score': 0.09017579257488251}]}
idx: 3670, nct_id: NCT00668525, prediction: 0.09017579257488251, label: 0.03420752565564424
{'predictions': [{'score': 0.14179369807243347}]}
idx: 3671, nct_id: NCT01359449, prediction: 0.14179369807243347, label: 0.04878048780487805
{'predictions': [{'score': 1.457583189010620

KeyboardInterrupt: 

In [ ]:
def evaluate(linear_predictor, test_features, test_labels, model_name, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  Return binary classification metrics.
    """
    # split the test data set into 100 batches and evaluate using prediction endpoint
    prediction_batches = [linear_predictor.predict(batch)['predictions'] for batch in np.array_split(test_features, 100)]
    # parse raw predictions json to exctract predicted label
    test_preds = np.concatenate([np.array([x['predicted_label'] for x in batch]) for batch in prediction_batches])
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    f1 = 2 * precision * recall / (precision + recall)
    
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actuals'], colnames=['predictions']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('F1:', f1))
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 'Precision': precision, 'Recall': recall, 'Accuracy': accuracy, 
             'F1': f1, 'Model': model_name}

The predictor.predict method call takes one parameter, the input data for which you want the SageMaker Endpoint to provide inference. predict will serialize the input data, and send it in as request to the SageMaker Endpoint by an InvokeEndpoint SageMaker operation. InvokeEndpoint operation requests can be made by predictor.predict, by boto3 SageMaker.runtime client or by AWS CLI.